# Modell zur Bachelorarbeit

### Einrichtung

In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math

import pypsa
import pypsa.plot as bmplot

%matplotlib inline

## Setup

Einrichtung aller wichtigen Kompnenten und Einlesen der Daten

Die jeweiligen Daten entstammen einer anderen Bachelorarbeit und werden über die von pandas zur Verfügung gestellten Funktionen aus Excel-Dateien eingelesen und als Dataframe gespeichert. Aus diesem werden dann wiederum nur die benötigten Werte verwendet.

### Variablen

In [3]:
nw = pypsa.Network()

In [15]:
dates = pd.date_range("2015-11-01T04:00:00Z", periods=5, freq="4H")

nw.set_snapshots(dates)

### Einlesen der Datenbanken

In [5]:
cur_path = r"C:\Users\janni\OneDrive\TU Berlin\4 - Abschlussarbeiten\01 - Bachelorarbeit\modell\data\\"

Zuerst werden die zur Verfügung gestellten Daten eingelesen, danach werden diese noch um einige benötigte Werte von Hand erweitert, die zur Durchführung des Unit Commitment nötig sind

In [10]:
gen_data = [pd.DataFrame for i in range(2)]
ex = pd.ExcelFile(cur_path + "Technische Daten Gaskraftwerke.xlsx")

for i in range(len(gen_data)):
    # Extract last two columns from worksheet and only rows 5-12
    gen_data[i] = ex.parse(i).iloc[4:11,[-2,-1]]
    # Use values from first column as index
    gen_data[i].index = gen_data[i].iloc[:,0]
    # Delete column in the middle
    gen_data[i] = gen_data[i].iloc[:, -1]
    gen_data[i].set_value("commitable", True)

In [12]:
wind_data = pd.read_excel(cur_path + "Stromlast und Windeinspeisung.xlsx",
                    sheet_name="Tabelle1",
                    header=3,
                    index_col=0,
                    parse_cols="A:F")

# Extract only the first day, every Xh
intervall = 16
wind_data_modell = wind_data.iloc[intervall::intervall,[0,1]].head((96//intervall)-1)

In [18]:
wind_data_modell.ix[0]

C:\Users\janni\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



Last                        38435
Windeinspeisung Original     6388
Name: 2015-11-01T04:00:00Z, dtype: int64

In [33]:
wind_data_modell.index == dates

TypeError: Cannot change data-type for object array.

In [13]:
print(wind_data_modell)

                       Last  Windeinspeisung Original
Datum, Zeit                                          
2015-11-01T04:00:00Z  38435                      6388
2015-11-01T08:00:00Z  45193                      5370
2015-11-01T12:00:00Z  42061                      4379
2015-11-01T16:00:00Z  49538                      5165
2015-11-01T20:00:00Z  48412                      8393


### Temporary Additional code
### ================================================

In [9]:
filenames = ["vertices", "lines", "loads"]
units = ["Bus", "Line", "Load"]
components = [pd.DataFrame for i in range(len(filenames))]

In [10]:
for i in range(len(components)):
    components[i] = pd.read_csv(cur_path + filenames[i] + ".csv")
    components[i].fillna(0,inplace=True)

components[1].rename(columns={"wid":"x","len":"length","cap":"s_nom"},inplace=True)

In [13]:
wind_data_modell["Last"] = wind_data_modell["Last"]/8
gen_data[1][1] = gen_data[1][1]*3

### ================================================

In [8]:
input_data = {"vertices":components[0],
              "lines":components[1],
              "loads":components[2],
              "GuD":gen_data[0],
              "Gasturbine":gen_data[1]}

#for i in range(len(input_data)):
#    print("lala")

NameError: name 'components' is not defined

In [7]:
input_data["GuD"]

NameError: name 'input_data' is not defined

In [ ]:
pypsa.io.import_components_from_dataframe(nw, input_data["GuD"], "Generator")

Einlesen der als *.csv gespeicherten Daten in ein pandas DataFrame, alle NaN werden durch 0 ersetzt

for i in range(len(components)):
    components[i] = pd.read_csv(cur_path + filenames[i] + ".csv")
    components[i].fillna(0,inplace=True)

components[1].rename(columns={"wid":"x","len":"length","cap":"s_nom"},inplace=True)
components[2].rename(columns={"marg. Cost":"marginal_cost","min power":"p_min_pu","startup costs":"start_up_cost","nom power":"p_nom"},inplace=True)

Uebertragen der Daten von pandas in das Netzwerk

for i in range(len(components)):
    pypsa.io.import_components_from_dataframe(nw, components[i], units[i])

Einlesen der Werte pro Zeitintervall

In [ ]:
#nw.import_series_from_dataframe(pd.read_csv(cur_path + "load_dist2" + ".csv"),
#                                "Load",
#                                "p_set")

In [ ]:
df = pd.DataFrame(pd.read_csv(cur_path + "load_dist2" + ".csv"))
df.index = dates
nw.import_series_from_dataframe(df, "Load", "p_set")

## Optimierung

nw.lopf()

## Aufbereitung der Daten

gen_outs = nw.generators_t.p
caps_by_carrier = nw.generators.groupby("carrier")["p_nom"].sum()
gen_pot_remaining = nw.generators.p_nom - gen_outs
print(gen_outs)

#dates = pd.date_range("2017-01-01", periods=6, freq="4H")
#dates

gen_pot_remaining

## Visualization

diag_rows = len(nw.generators)
diag_cols = len(nw.snapshots)
diag_inches = 4

fig, axes = plt.subplots(nrows=diag_rows, ncols=diag_cols)
fig.set_size_inches(diag_inches*diag_cols, diag_inches*diag_cols)

for i in range(diag_rows):
    for j in range(diag_cols):
        perc_labels = ["Available", "Dispatched"]
        sizes = [gen_pot_remaining.loc[dates[j],"{0}".format(i)],
                 gen_outs.loc[dates[j],"{0}".format(i)]]
        colors = ["green", "blue"]

        axes[i,j].pie(sizes, labels=perc_labels, colors=colors, autopct="%1.1f%%")
        axes[i,j].axis("equal")
        axes[0,j].set_title(dates[j])
        axes[i,0].set_ylabel(components[2]["name"][i])
        plt.subplots_adjust(wspace=0.7, hspace=0.2)
        #plt.pie(fig, labels=perc_labels, colors=colors, autopct="%1.1f%%")
        #plt.axis("equal")

workload_rows = len(nw.generators)
workload_inches = 4

# 3 columns to 
fig, axes = plt.subplots(nrows=math.ceil(workload_rows/3), ncols=3)
fig.set_size_inches(diag_inches*diag_cols, diag_inches*diag_cols)

Line graph to display the dispatched amount of energy per plant

gen_plot = gen_outs.plot(kind="area",grid=True)
lines, labels = gen_plot.get_legend_handles_labels()
gen_plot.legend(lines[:len(nw.generators)], components[2]['name'].values.tolist(), loc='best')
plt.ylabel("MW")

network graph showing the distribution of buses etc.

bmplot.plot(nw, title="Aufbau Netz")
plt.xlim([0,120])
plt.ylim([0,120])